# Step9, 함수를 더 편리하게 

역전파를 해결 할수 있었다.  
Define-by-Run이라는 전체 계산의 각 조각들을 런타임에 '연결'도 하였다.  

하지만 사용하기에 조금 불편한 부분이 있어서 세가지를 개선한다.

## 9.1 파이썬 함수로 이용하기

In [31]:
from function import Variable, Square, Exp
import numpy as np 

x = Variable(np.array(0.5))
f = Square()
y = f(x)

Square 클래스의 인스턴스를 생성한 다음 이어서 그 인스턴스를 호출하는 두 단계로 구분해 진행 

번거롭 

첫번째 개선 : '파이썬 함수'를 지원하는 방법

In [32]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [33]:
x = Variable(np.array(0.5))

a = square(x)
b = exp(a)
y = square(b)
print(y.data)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

1.648721270700128
3.297442541400256


In [34]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## 9.2 backward 메서드 간소화

두번째 개선: 역전파 시 사용자의 번거로움 줄이기

y.grad = np.array(1.0) 부부을 생략하기

In [42]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def square(x):
    return Square()(x)


def exp(x):
    return Exp()(x)



**Note_** 이전까지 출력의 미분값을 np.array(1.0)으로 사용 np.ones_like()를 사용  
이유 : Variable의 data와 grad의 데이터 타입을 같게 만들기 위해

In [43]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


## 9.3 ndarray만 취급하기

Variable은 데이터로 ndarray 인스턴스만 취급하려는 의도.  
하지만 float나 int 데이터 타입 사용  
이런 사태를 막기위해 Variable이 ndarray 인스턴스만 담는 '상자'가 되도록 고민  
그래서 Variable에 ndarray 인스턴스 외에 데이터를 넣을 경우 즉시 오류 발생

In [45]:
class Variable:
    def __init__(self, data):
        if data is not None:                            # 데이터가 None은 아닌데
            if not isinstance(data, np.ndarray):        # ndarray도 아니면      오류 발생
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        
        self.data = data
        self.grad = None 
        self.creator = None 

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)
                

In [48]:
x = Variable(np.array(1.0))     # ok
x = Variable(None)              # ok 

In [49]:
x = Variable(1.0)               # 오류

TypeError: <class 'float'>은(는) 지원하지 않습니다.

그런데 이렇게 바꾸면 주의할 게 하나 생깁니다.

In [52]:
x = np.array([1.0])
y = x ** 2

print(x)
print(y)

print(type(x), x.ndim)
print(type(y))
# x는 1차원 ndarray이다. 여기에 제곱을 하면 y의 데이터 타입도 ndarray이다.
# 문제가 되는 것은 다음의 경우이다.

[1.]
[1.]
<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [53]:
x = np.array(1.0)
y = x ** 2

print(x)
print(y)

print(type(x), x.ndim)
print(type(y))


1.0
1.0
<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


여기서 x는 0차원의 ndarray인데, 제곱(x**2)을 하면 np.float64가 되어버린다.  
넘파이가 의도한 동작이다.  
즉, 0차원 ndarray 인스턴스를 사용하여 계산하면 결과의 데이터 타입이 numpy.float64나 numpy.float32 등으로 달라진다!!

따라서 Variable은 데이터가 항상 ndarray 인스턴스라고 가정하고 있으므로 이 부분을 해결해야한다.

In [55]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)      # 스칼라이면 array로 바꿔서 리턴
    return x                    # 스칼라 아니면(array이면) 그대로 리턴

np.isscalar(x) : 입력 데이터 x가 numpy.float64 같은 스칼라 타입인지를 확인해주는 함수 (파이썬에서 int타입,float타입 --> 스칼라)

Function 클래스에 as_array()를 추가해준다.

In [56]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

이와 같이 순전파의 결과인 y를 Variable로 감쌀 때 as_array()를 이용  
이렇게 하여 출력 결과인 output은 항상 ndarray 인스턴스가 되도록 보장

지금까지 최종 코드

In [58]:
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:                            # 데이터가 None은 아닌데
            if not isinstance(data, np.ndarray):        # ndarray도 아니면      오류 발생
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        
        self.data = data
        self.grad = None 
        self.creator = None 

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)
                

def as_array(x):
    if np.isscalar(x):
        return np.array(x)      # 스칼라이면 array로 바꿔서 리턴
    return x                    # 스칼라 아니면(array이면) 그대로 리턴

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def square(x):
    return Square()(x)


def exp(x):
    return Exp()(x)

NameError: name '드' is not defined